In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()


1) Прочитайте файл alerts.json

In [3]:
df = spark.read.format('json').load('alerts.json')
df.show()

+-----------+--------------------+-------------------+-------------------+--------------------+----------+
|   alert_id|     entitled_assets|  event_source_type|         event_time|         instance_id|    serial|
+-----------+--------------------+-------------------+-------------------+--------------------+----------+
|ArbwAO2m4Oa|  [qnTrje2, WT87cRS]|healthchecker_alert|2019-06-07 20:50:41|ypJdMVE8XfRgbq8E2...|cyN-1QeXWm|
|1AlMxb4jeZ8|[VrE3Mr8, Z0SBQ7L...|healthchecker_alert|2019-06-11 12:46:18|Aygpbf1YPEpvn1CQj...|Ei3-SNCq7e|
|ArbwAO2m4Oa|           [MWg0cOi]|       expire_alert|2019-06-15 13:08:36|QcEbVRV6D5KCQsQ9L...|cyo-8FZaon|
|KFto5Cml9aH|           [OawSMoe]|       expire_alert|2019-06-02 19:23:49|nTZsmuJWP39TgmoVG...|IfS-5Qrifc|
|MlsUENcq3iO|[x8BcRNi, QCDZz5u...|         case_event|2019-06-13 23:45:41|tklUIehgbpBYLIvbo...|DXq-WmHlJj|
|u6GnQ51znX6|[KIAO8jQ, RzWUoRl...|       expire_alert|2019-06-19 00:15:42|5qkV4UyYQ596ZrG4M...|bc8-5A2FiC|
|jlcYc5PQ5u5|[Za1BpUf, 7lVpL0Q...|hea

2) Выведите схему

In [4]:
df.printSchema()

root
 |-- alert_id: string (nullable = true)
 |-- entitled_assets: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- event_source_type: string (nullable = true)
 |-- event_time: string (nullable = true)
 |-- instance_id: string (nullable = true)
 |-- serial: string (nullable = true)



3) Выберите данные c 2019-06-10 по 2019-06-19. Дата и время хранятся в атрибуте event_time.

In [5]:
tmp_df = df.withColumn("event_time", to_timestamp(df.event_time))
task3 = tmp_df.filter(tmp_df.event_time >= lit("2019-06-10")).filter(tmp_df.event_time < lit("2019-06-19"))
task3.show()

+-----------+--------------------+-------------------+-------------------+--------------------+----------+
|   alert_id|     entitled_assets|  event_source_type|         event_time|         instance_id|    serial|
+-----------+--------------------+-------------------+-------------------+--------------------+----------+
|1AlMxb4jeZ8|[VrE3Mr8, Z0SBQ7L...|healthchecker_alert|2019-06-11 12:46:18|Aygpbf1YPEpvn1CQj...|Ei3-SNCq7e|
|ArbwAO2m4Oa|           [MWg0cOi]|       expire_alert|2019-06-15 13:08:36|QcEbVRV6D5KCQsQ9L...|cyo-8FZaon|
|MlsUENcq3iO|[x8BcRNi, QCDZz5u...|         case_event|2019-06-13 23:45:41|tklUIehgbpBYLIvbo...|DXq-WmHlJj|
|oIpXWTYPGOs|[yFnOS8v, r9RY2sq...|         case_event|2019-06-12 00:08:47|jEx38fGy7dJTQ9Gad...|n0m-gP8Smi|
|O9zC9Tx6eJv|           [gllPqNY]|         case_event|2019-06-11 08:46:38|O8RCU305KiPu12mbm...|5y6-7asaXz|
|ChPjT8d8BRX|           [ybGo8YS]|         case_event|2019-06-18 08:52:06|smtMLyJZIpZ9MIK3U...|8sR-QXaV8Y|
|05exWW0L1LJ|[OSWlI3D, aiie3db...|   

4) Выберите данные типа healthchecker_alert (атрибут event_source_type)

In [6]:
task4 = task3.filter(df.event_source_type == "healthchecker_alert")
task4.show()

+-----------+--------------------+-------------------+-------------------+--------------------+----------+
|   alert_id|     entitled_assets|  event_source_type|         event_time|         instance_id|    serial|
+-----------+--------------------+-------------------+-------------------+--------------------+----------+
|1AlMxb4jeZ8|[VrE3Mr8, Z0SBQ7L...|healthchecker_alert|2019-06-11 12:46:18|Aygpbf1YPEpvn1CQj...|Ei3-SNCq7e|
|oIpXWTYPGOs|  [6YaSkE7, eZj0nFj]|healthchecker_alert|2019-06-13 15:52:00|iYEqgyBhCEiyln10m...|7oC-nERLNr|
|XTBrpBEIW2v|[O9aK1Rp, dj11St9...|healthchecker_alert|2019-06-15 19:31:46|ipTaSHDtk4uVRUQra...|MPe-eo4QyM|
|9eWpr1cPXEE|[jYHGoRE, qFZBHOX...|healthchecker_alert|2019-06-18 16:46:09|1RhDsVckpfNrVeXbw...|2to-bdeDHk|
|mEmTZVPrTJd|[NphC1wS, lBmbcwL...|healthchecker_alert|2019-06-15 15:23:07|O2lSEoEkxZcmxZCH5...|2sb-e558sZ|
|czQwIHOzm03|[WbsKFGu, ekG3XXE...|healthchecker_alert|2019-06-12 01:10:03|eoSV6vRsEK0lrfQjd...|x23-uhxUOj|
|BxKRvWMrY6I|[DcVSAKy, AJDUVsf...|hea

5) Для каждого alert_id оставьте только первое по времени событие за каждый день.

In [7]:
tmp_df = task4.groupBy("alert_id", to_date("event_time").alias("date")).agg(min("event_time").alias("event_time_min"))
task5 = task4.drop("alert_id").join(tmp_df, (tmp_df.event_time_min == task4.event_time), 'inner').drop('event_time_min', 'date')
task5.show()

+--------------------+-------------------+-------------------+--------------------+----------+-----------+
|     entitled_assets|  event_source_type|         event_time|         instance_id|    serial|   alert_id|
+--------------------+-------------------+-------------------+--------------------+----------+-----------+
|[VrE3Mr8, Z0SBQ7L...|healthchecker_alert|2019-06-11 12:46:18|Aygpbf1YPEpvn1CQj...|Ei3-SNCq7e|1AlMxb4jeZ8|
|[jYHGoRE, qFZBHOX...|healthchecker_alert|2019-06-18 16:46:09|1RhDsVckpfNrVeXbw...|2to-bdeDHk|9eWpr1cPXEE|
|[WbsKFGu, ekG3XXE...|healthchecker_alert|2019-06-12 01:10:03|eoSV6vRsEK0lrfQjd...|x23-uhxUOj|czQwIHOzm03|
|[DcVSAKy, AJDUVsf...|healthchecker_alert|2019-06-16 11:55:05|R78fLiC3uzvCGwRzV...|YAU-D9IQ4R|BxKRvWMrY6I|
|[IClkMK8, 6XSFoXM...|healthchecker_alert|2019-06-15 03:48:21|w8DFnQMsmvSRItCKy...|WES-YQrlKS|vliWc3PkkOJ|
|           [kPao2qc]|healthchecker_alert|2019-06-11 09:44:43|9JMrTtAi6StVb7c2y...|GyG-DDnIsH|YqxeNT2b1WJ|
|[gbVdL74, CGcQHu6...|healthchecker_a

6) В поле entitled_assets хранится массив данных. Размножьте каждую строку таким образом, чтобы в новой строке было одно значение из этого массива. 

In [8]:
task6 = task5.select(task5.alert_id, task5.event_source_type, explode(task5.entitled_assets).alias('asset_id') , task5.event_time, task5.instance_id, task5.serial)
task6.show()

+-----------+-------------------+--------+-------------------+--------------------+----------+
|   alert_id|  event_source_type|asset_id|         event_time|         instance_id|    serial|
+-----------+-------------------+--------+-------------------+--------------------+----------+
|1AlMxb4jeZ8|healthchecker_alert| VrE3Mr8|2019-06-11 12:46:18|Aygpbf1YPEpvn1CQj...|Ei3-SNCq7e|
|1AlMxb4jeZ8|healthchecker_alert| Z0SBQ7L|2019-06-11 12:46:18|Aygpbf1YPEpvn1CQj...|Ei3-SNCq7e|
|1AlMxb4jeZ8|healthchecker_alert| aASolix|2019-06-11 12:46:18|Aygpbf1YPEpvn1CQj...|Ei3-SNCq7e|
|9eWpr1cPXEE|healthchecker_alert| jYHGoRE|2019-06-18 16:46:09|1RhDsVckpfNrVeXbw...|2to-bdeDHk|
|9eWpr1cPXEE|healthchecker_alert| qFZBHOX|2019-06-18 16:46:09|1RhDsVckpfNrVeXbw...|2to-bdeDHk|
|9eWpr1cPXEE|healthchecker_alert| q5MY47C|2019-06-18 16:46:09|1RhDsVckpfNrVeXbw...|2to-bdeDHk|
|9eWpr1cPXEE|healthchecker_alert| 8EmgEx0|2019-06-18 16:46:09|1RhDsVckpfNrVeXbw...|2to-bdeDHk|
|czQwIHOzm03|healthchecker_alert| WbsKFGu|2019-06-

7) Оставьте только следующие колонки: alert_id, event_time, date, asset_id. Получившийся датафрейм сохраните в формате csv. 

In [9]:
task7 = task6.select(task6.alert_id, task6.event_time, to_date(task6.event_time).alias("date"), task6.asset_id).sort(col("event_time"))
task7.show()

+-----------+-------------------+----------+--------+
|   alert_id|         event_time|      date|asset_id|
+-----------+-------------------+----------+--------+
|HttiH0rqiG4|2019-06-10 00:25:33|2019-06-10| qN24iyB|
|HttiH0rqiG4|2019-06-10 00:25:33|2019-06-10| KKERafq|
|HttiH0rqiG4|2019-06-10 00:25:33|2019-06-10| MMlFGvu|
|XTBrpBEIW2v|2019-06-10 00:26:25|2019-06-10| SCXC3cX|
|XTBrpBEIW2v|2019-06-10 00:26:25|2019-06-10| 6P6x5Ap|
|XTBrpBEIW2v|2019-06-10 00:26:25|2019-06-10| pSGYvkI|
|XTBrpBEIW2v|2019-06-10 00:26:25|2019-06-10| 4rDvwJ2|
|XTBrpBEIW2v|2019-06-10 00:26:25|2019-06-10| S42ucE0|
|VRYMEmmYp50|2019-06-10 00:33:47|2019-06-10| cSgmgGz|
|VRYMEmmYp50|2019-06-10 00:33:47|2019-06-10| 6aMVSiH|
|VRYMEmmYp50|2019-06-10 00:33:47|2019-06-10| H8vYSwJ|
|VRYMEmmYp50|2019-06-10 00:33:47|2019-06-10| 3Z2st9i|
|VRYMEmmYp50|2019-06-10 00:33:47|2019-06-10| Odgrajz|
|YqxeNT2b1WJ|2019-06-10 00:53:26|2019-06-10| W23dmp5|
|YqxeNT2b1WJ|2019-06-10 00:53:26|2019-06-10| PCKkjgY|
|aGYNCmS1qwf|2019-06-10 01:0

In [10]:
# task7.coalesce(1).write.csv('result.csv')
task7.coalesce(1)\
     .write.format("com.databricks.spark.csv")\
     .option("header", "true")\
     .save("result.csv")

8*) Проделайте те же преобразования с помощью sql-запроса и метода .sql()

In [11]:
spark.stop()